In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 37630673
paper_name = 'grosjean_blaudez_2023' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/microorganisms-2508478-supplementary.xlsx', 
                              sheet_name='Supplemental Table S1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 414 x 9


In [10]:
original_data.head()

,SGD ID,ORF,ORF name,ORF name description,Gd,Description,Human ortholog,Human ortholog description,OMIM disease associated
0,S000000331,YBR127C,VMA2,Vacuolar Membrane Atpase,MS,Subunit B of V1 peripheral membrane domain of ...,"ATP6V1B2,ATP6V1B1","ATPase, H+ transporting, lysosomal 56/58kDa, V...","ZIMMERMANN-LABAND SYNDROME 2; ZLS2,RENAL TUBUL..."
1,S000002980,YGL012W,ERG4,ERGosterol biosynthesis,MS,C-24(28) sterol reductase; catalyzes the final...,DHCR7,7-dehydrocholesterol reductase,SMITH-LEMLI-OPITZ SYNDROME; SLOS
2,S000005378,YOL018C,TLG2,T-snare affecting a Late Golgi compartment,MS,Syntaxin-like t-SNARE; forms a complex with Tl...,STX16,syntaxin 16,"PSEUDOHYPOPARATHYROIDISM, TYPE IB; PHP1B"
3,S000000623,YCR028C,FEN2,FENpropimorph resistance,MS,Plasma membrane H+-pantothenate symporter; con...,"SLC17A9,SLC17A5,SLC17A5",solute carrier family 17 (vesicular nucleotide...,"POROKERATOSIS 8, DISSEMINATED SUPERFICIAL ACTI..."
4,S000003440,YGR208W,SER2,SERine requiring,MS,Phosphoserine phosphatase of the phosphoglycer...,PSPH,phosphoserine phosphatase,PHOSPHOSERINE PHOSPHATASE DEFICIENCY; PSPHD


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [SGD ID, ORF, ORF name, ORF name description, Gd, Description, Human ortholog, Human ortholog description, OMIM disease associated, orf]
Index: []


In [15]:
data_map = {'HS':-3,'MS':-2,'LS':-1,'HR':3,'MR':2,'LR':1}

In [16]:
original_data['data'] = original_data['Gd'].apply(lambda x: data_map[x])

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['data']].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(414, 1)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [22265]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,22265
data_type,value
orf,
YAL016W,1
YAR020C,1
YBL006C,-1
YBL007C,2
YBL012C,-1


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22265
,data_type,value
gene_id,orf,
14,YAL016W,1
70,YAR020C,1
94,YBL006C,-1
95,YBL007C,2
100,YBL012C,-1


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22265          
data_type       value    valuez
gene_id orf                    
14      YAL016W     1  1.699875
70      YAR020C     1  1.699875
94      YBL006C    -1 -1.806117
95      YBL007C     2  3.452871
100     YBL012C    -1 -1.806117

# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')